In [80]:
import numpy as np
from core.utils import load_metrics

In [81]:
burma_path = './dataset/burma14.xml'

In [82]:
weight_metric = load_metrics(burma_path)
weight_metric.shape

d:\USERS\Exeter\Courses\Module ECMM409 (2023) Nature-Inspired Computation\assessments\project01\core\utils.py:16: RuntimeWarning: invalid value encountered in multiply
  weights_metric = np.nan_to_num(np.identity(num_node) * -np.inf, 0.0)


(14, 14)

In [106]:
pop_size = 10
tournament_size = 4
crossover_rate = 0.90
mutate_rate = 0.20



In [84]:
# generate population
def create_candidate(distance_metric):
    return list(np.random.permutation(len(distance_metric)))

def generate_population(distance_metric, pop_size:int=10):
    return [create_candidate(distance_metric) for _ in range(pop_size)]


In [85]:
pop1 = generate_population(weight_metric)
pop1

[[11, 4, 1, 12, 7, 10, 5, 3, 0, 13, 2, 9, 8, 6],
 [11, 3, 2, 0, 13, 7, 1, 4, 9, 12, 8, 10, 6, 5],
 [9, 8, 13, 7, 10, 0, 2, 12, 1, 6, 5, 4, 11, 3],
 [13, 10, 3, 5, 12, 1, 7, 11, 0, 2, 6, 8, 9, 4],
 [4, 11, 12, 3, 5, 10, 1, 8, 6, 2, 9, 7, 13, 0],
 [2, 10, 8, 4, 5, 13, 3, 1, 9, 7, 12, 6, 11, 0],
 [11, 2, 5, 12, 10, 8, 6, 3, 1, 4, 9, 0, 7, 13],
 [4, 10, 5, 9, 2, 13, 11, 3, 0, 6, 12, 7, 8, 1],
 [11, 7, 10, 9, 0, 4, 5, 2, 12, 1, 13, 6, 8, 3],
 [0, 8, 2, 5, 7, 12, 3, 11, 10, 4, 9, 13, 6, 1]]

In [86]:
cand1 = create_candidate(weight_metric)
cand1

[4, 12, 1, 11, 8, 2, 10, 9, 7, 6, 0, 5, 3, 13]

In [87]:
def cost_function(travel_list, distance_metric):
    total_distance = 0
    for i in range(len(travel_list)-1):
        total_distance += distance_metric[travel_list[i],travel_list[i+1]]
    total_distance += distance_metric[travel_list[-1],travel_list[0]]
    return total_distance

def fitness(travel_list, distance_metric):
    return 1 / cost_function(travel_list, distance_metric)

In [88]:
cost1 = cost_function(cand1, weight_metric)
cost1

6879.0

In [89]:
fitness1 = fitness(cand1, weight_metric)
fitness1

0.00014536996656490768

In [90]:
[fitness(can, weight_metric) for can in pop1]

[0.00016097875080489375,
 0.00016874789065136686,
 0.00017784101013693757,
 0.0001414027149321267,
 0.00015163002274450342,
 0.00017283097131005876,
 0.0001603077909586406,
 0.000142025280499929,
 0.0001636125654450262,
 0.00013906271728549575]

In [91]:
pop_fitness = [fitness(can, weight_metric) for can in pop1]
pop_fitness

[0.00016097875080489375,
 0.00016874789065136686,
 0.00017784101013693757,
 0.0001414027149321267,
 0.00015163002274450342,
 0.00017283097131005876,
 0.0001603077909586406,
 0.000142025280499929,
 0.0001636125654450262,
 0.00013906271728549575]

In [92]:
max(pop_fitness)

0.00017784101013693757

In [93]:
pop_dict = {
    "population" : pop1,
    "fitness" : pop_fitness,
}

In [94]:
pop_dict['population'][0] , pop_dict['fitness'][0]

([11, 4, 1, 12, 7, 10, 5, 3, 0, 13, 2, 9, 8, 6], 0.00016097875080489375)

In [109]:
for i in pop_dict.values():
    print(i)
    break


[[11, 4, 1, 12, 7, 10, 5, 3, 0, 13, 2, 9, 8, 6], [11, 3, 2, 0, 13, 7, 1, 4, 9, 12, 8, 10, 6, 5], [9, 8, 13, 7, 10, 0, 2, 12, 1, 6, 5, 4, 11, 3], [13, 10, 3, 5, 12, 1, 7, 11, 0, 2, 6, 8, 9, 4], [4, 11, 12, 3, 5, 10, 1, 8, 6, 2, 9, 7, 13, 0], [2, 10, 8, 4, 5, 13, 3, 1, 9, 7, 12, 6, 11, 0], [11, 2, 5, 12, 10, 8, 6, 3, 1, 4, 9, 0, 7, 13], [4, 10, 5, 9, 2, 13, 11, 3, 0, 6, 12, 7, 8, 1], [11, 7, 10, 9, 0, 4, 5, 2, 12, 1, 13, 6, 8, 3], [0, 8, 2, 5, 7, 12, 3, 11, 10, 4, 9, 13, 6, 1]]


In [110]:
for i , (pop, fit) in enumerate(pop_dict.items()):
    print(i, pop, fit)
    break

0 population [[11, 4, 1, 12, 7, 10, 5, 3, 0, 13, 2, 9, 8, 6], [11, 3, 2, 0, 13, 7, 1, 4, 9, 12, 8, 10, 6, 5], [9, 8, 13, 7, 10, 0, 2, 12, 1, 6, 5, 4, 11, 3], [13, 10, 3, 5, 12, 1, 7, 11, 0, 2, 6, 8, 9, 4], [4, 11, 12, 3, 5, 10, 1, 8, 6, 2, 9, 7, 13, 0], [2, 10, 8, 4, 5, 13, 3, 1, 9, 7, 12, 6, 11, 0], [11, 2, 5, 12, 10, 8, 6, 3, 1, 4, 9, 0, 7, 13], [4, 10, 5, 9, 2, 13, 11, 3, 0, 6, 12, 7, 8, 1], [11, 7, 10, 9, 0, 4, 5, 2, 12, 1, 13, 6, 8, 3], [0, 8, 2, 5, 7, 12, 3, 11, 10, 4, 9, 13, 6, 1]]


In [96]:
pop1[0] , pop1[1]

([11, 4, 1, 12, 7, 10, 5, 3, 0, 13, 2, 9, 8, 6],
 [11, 3, 2, 0, 13, 7, 1, 4, 9, 12, 8, 10, 6, 5])

### Selection

In [112]:
np.random.choice([pop1])

ValueError: a must be 1-dimensional

In [115]:
def tournament_selection(population, tournament_size):
    fit_value = -np.inf
    res = None
    for _ in range(tournament_size):
        idx = np.random.randint(len(population))
        new_fit = fitness(population[idx], distance_metric=weight_metric)
        if new_fit >= fit_value:
            fit_value = new_fit
            res = population[idx]
        
    return res

In [128]:
tournament_selection(pop1, 4), tournament_selection(pop1, 4)

([11, 3, 2, 0, 13, 7, 1, 4, 9, 12, 8, 10, 6, 5],
 [11, 7, 10, 9, 0, 4, 5, 2, 12, 1, 13, 6, 8, 3])

### Operation

In [101]:
from typing import Optional

n=14
def crossover(arg1, arg2, locus:Optional[int]=None):
    if locus is None:
        locus = np.random.randint(n-1)
    child1 = arg1[:locus] + arg2[locus:] 
    child2 = arg2[:locus] + arg1[locus:]
    return child1, child2

def mutate(arg):
    """ mutate (swap operator)
        swapping between 2 indexes

    Args:
        arg (_type_): _description_
        mutate_rate (_type_): _description_

    Returns:
        crossover's child (List(int)): _description_ 
    """
    idx1, idx2 = np.random.randint(n-1,size=2)
    res = arg
    res[idx1], res[idx2] = res[idx2], res[idx1]
    return res




In [105]:
child1, child2 = crossover(pop1[0],pop1[1], 14)
child1, child2

([11, 4, 1, 12, 7, 10, 5, 3, 0, 13, 2, 9, 8, 6],
 [11, 3, 2, 0, 13, 7, 1, 4, 9, 12, 8, 10, 6, 5])

### Replacement

In [ ]:
def replace_1stweak(pop1, childs):
    temp_pop = []
    for child in childs:
        res = []
        fit = fitness(child)
        for cand in pop1:
            if fit >= fitness(cand):



In [104]:
pop1

[[11, 4, 1, 12, 7, 10, 5, 3, 0, 13, 2, 9, 8, 6],
 [11, 3, 2, 0, 13, 7, 1, 4, 9, 12, 8, 10, 6, 5],
 [9, 8, 13, 7, 10, 0, 2, 12, 1, 6, 5, 4, 11, 3],
 [13, 10, 3, 5, 12, 1, 7, 11, 0, 2, 6, 8, 9, 4],
 [4, 11, 12, 3, 5, 10, 1, 8, 6, 2, 9, 7, 13, 0],
 [2, 10, 8, 4, 5, 13, 3, 1, 9, 7, 12, 6, 11, 0],
 [11, 2, 5, 12, 10, 8, 6, 3, 1, 4, 9, 0, 7, 13],
 [4, 10, 5, 9, 2, 13, 11, 3, 0, 6, 12, 7, 8, 1],
 [11, 7, 10, 9, 0, 4, 5, 2, 12, 1, 13, 6, 8, 3],
 [0, 8, 2, 5, 7, 12, 3, 11, 10, 4, 9, 13, 6, 1]]